In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/bookgenrenlproben1/trainnlp01.csv')
df

In [ ]:
df.genre.value_counts()

In [ ]:
import tensorflow as tf

In [ ]:
df.head()

In [ ]:
unique_label=df.genre.unique()
unique_label

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
gdata = {
    'Genre': ['Fiction', 'Thriller', 'Childrens-Book', 'Political', 'Science-Fiction', 'War', 'Motivational'],
    'Label': [0, 1, 2, 3, 4, 5, 6]
}

genre_df = pd.DataFrame(gdata)

genre_df

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)
df.isna().sum()

In [ ]:
df.Label.value_counts().plot(kind='bar');


In [ ]:
df.summary.sample()

In [ ]:
index = np.random.randint(0, len(df))

print(index)
df['summary'][index]

In [ ]:
print(index)

In [ ]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import string
import re

In [ ]:
def plot_world(text):

    wordcloud = WordCloud(width = 500, height = 500, 
                    background_color ='black', 
                    font_path="Nikosh.ttf",
                    min_font_size = 10).generate(text) 

                            
    plt.figure(figsize = (5, 5), facecolor = 'k', edgecolor = 'k' ) 
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 

    plt.show()

In [ ]:
!wget https://www.easynepalityping.com/resource/font/bangla/06-nikosh-bangla-font.zip
!unzip 06-nikosh-bangla-font.zip    

In [ ]:
s= " ".join(df.summary[:100])
plot_world(s)

In [ ]:
pip install bnlp-toolkit

In [ ]:

from bnlp import NLTKTokenizer
tokenizer = NLTKTokenizer()

df['summary'] = df['summary'].apply(lambda x: tokenizer.word_tokenize(str(x)) if pd.notna(x) else x)


In [ ]:
from bnlp import CleanText

cleaner = CleanText(
    fix_unicode=True,
    unicode_norm=True,
    unicode_norm_form="NFKC",
    remove_url=False,
    remove_email=False,
    remove_emoji=False,
    remove_number=False,
    remove_digits=False,
    remove_punct=False,
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_number="<NUMBER>",
    replace_with_digit="<DIGIT>",
    replace_with_punct="<PUNC>"
)


df['summary'] = df['summary'].apply(cleaner)
df.summary

In [ ]:
def remove_unicode_character(text, character):
    if pd.isna(text):
        return text
    return text.replace(character, '')

def remove_emojis(text):
    if pd.isna(text):
        return text
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F700-\U0001F77F"
                               u"\U0001F780-\U0001F7FF"
                               u"\U0001F800-\U0001F8FF"
                               u"\U0001F900-\U0001F9FF"
                               u"\U0001FA00-\U0001FA6F"
                               u"\U0001FA70-\U0001FAFF"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_carriage_return_newline(text):
    if pd.isna(text):
        return text
    return text.replace('\r\n', ' ')
def remove_punctuation(text):
    
    if pd.isna(text):
        return text

    
    translator = str.maketrans('', '', string.punctuation)
    
    
    return text.translate(translator)
unicode_character = '।'
df['summary'] = df['summary'].apply(remove_unicode_character, character=unicode_character)
df['summary'] = df['summary'].apply(remove_emojis)
df['summary'] = df['summary'].apply(remove_carriage_return_newline)
df['summary'] = df['summary'].apply(remove_punctuation)
df['summary'] = df['summary'].str.replace('BookReview', '')

In [ ]:
words_to_remove = ['ও', 'করে', 'এবং', 'এই', 'না', 'বইটি', 'করতে', 'থেকে', 'জন্য', 'এই',
                       'তার', 'করে', 'এক', 'আর', 'থেকে', 'সে', 'একটা', 'এর', 'এবং', 'ও', 'এই',
                       'তিনি', 'করে', 'এবং', 'তার', 'থেকে', 'আর', 'যে', 'বইটি', 'তার', 'এই', 'করে',
                       'আর', 'সে', 'না', 'যে', 'এই', 'তার', 'এক', 'আর', 'সে', 'থেকে', 'না', 'এই','অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য',
             'অবধি', 'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে',
             'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও',
             'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা',
             'এই', 'একই', 'একটি', 'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই',
             'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন',
             'এমনকী', 'এমনি', 'এর', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও','ওঁদের', 'ওঁর', 'ওঁরা',
             'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক',
             'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন', 'করলে', 'করলেন', 'করা',
             'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই',
             'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে',
             'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই',
             'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও', 'কোনো', 'ক্ষেত্রে', 'কয়েক', 'খুব', 'গিয়ে',
             'গিয়েছে','থাকে', 'থাকেন', 'থেকে', 'থেকেই', 'থেকেও', 'দিকে', 'দিতে', 'দিন', 'দিয়ে', 'দিয়েছে',
                    'দিয়েছেন', 'দিলেন', 'দু', 'দুই', 'দুটি', 'দুটো', 'দেওয়া', 'দেওয়ার', 'দেওয়া', 'দেখতে',
                    'দেখা', 'দেখে', 'দেন', 'দেয়', 'দ্বারা', 'ধরা', 'ধরে', 'ধামার', 'নতুন', 'নয়', 'না',
                    'নাই', 'নাকি', 'নাগাদ', 'নানা', 'নিজে', 'নিজেই', 'নিজেদের', 'নিজের', 'নিতে', 'নিয়ে',
                    'নিয়ে', 'নেই', 'নেওয়া', 'নেওয়ার', 'নেওয়া', 'নয়', 'পক্ষে', 'পর', 'পরে', 'পরেই',
                    'পরেও', 'পর্যন্ত', 'পাওয়া', 'পাচ', 'পারি', 'পারে', 'পারেন', 'পি', 'পেয়ে', 'পেয়্র্',
                    'প্রতি', 'প্রথম', 'প্রভৃতি', 'প্রযন্ত', 'প্রাথমিক', 'প্রায়', 'প্রায়', 'ফলে', 'ফিরে',
                    'ফের', 'বক্তব্য', 'বদলে', 'বন', 'বরং', 'বলতে', 'বলল', 'বললেন', 'বলা', 'বলে', 'বলেছেন',
                    'বলেন', 'বসে', 'বহু', 'বা', 'বাদে', 'বার', 'বি', 'বিনা', 'বিভিন্ন', 'বিশেষ', 'বিষয়টি',
                    'বেশ', 'বেশি', 'ব্যবহার', 'ব্যাপারে', 'ভাবে', 'ভাবেই', 'মতো', 'মতোই', 'মধ্যভাগে', 'মধ্যে',
                    'মধ্যেই', 'মধ্যেও', 'মনে', 'মাত্র', 'মাধ্যমে', 'মোট', 'মোটেই', 'যখন', 'যত', 'যতটা', 'যথেষ্ট',
                    'যদি', 'যদিও', 'যা', 'যাঁর', 'যাঁরা', 'যাওয়া', 'যাওয়ার', 'যাওয়া', 'যাকে', 'যাচ্ছে', 'যাতে',
                    'যাদের', 'যান', 'যাবে', 'যায়', 'যার', 'যারা', 'যিনি', 'যে', 'যেখানে', 'যেতে', 'যেন', 'যেমন',
                    'র', 'রকম', 'রয়েছে', 'রাখা', 'রেখে', 'লক্ষ', 'শুধু', 'শুরু', 'সঙ্গে', 'সঙ্গেও', 'সব', 'সবার',
                    'সমস্ত', 'সম্প্রতি', 'সহ', 'সহিত', 'সাধারণ', 'সামনে', 'সি', 'সুতরাং', 'সে', 'সেই', 'সেখান', 'সেখানে',
                    'সেটা', 'সেটাই', 'সেটাও', 'সেটি', 'স্পষ্ট', 'স্বয়ং', 'হইতে', 'হইবে','সাথে', '।', 'হয়ে', 'বই', 'লেখক', 'না।', 'তবে', 'শেষ', 'ভালো', 'যায়',
    '।', 'সাথে', 'হয়ে', 'নাম', 'কথা', 'যায়', 'তাঁর', '।', 'না।', 'সালে',
    'লেখক', 'সাথে', 'কথা', '।', 'হয়ে', 'সাথে', 'না।', 'মানুষ', 'মানুষের',
    'বই', 'লেখক', 'বই', 'হয়ে', 'সাথে', 'না।', 'বাংলাদেশের', 'তাঁর','অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি', 'অবশ্য', 'অর্থাত',
    'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার',
    'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই',
                  'একটি', 'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই', 'এতে', 'এদের',
    'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই',
    'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে',
    'করবেন', 'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই', 'করেছিলেন',
    'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে', 'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু',
    'কী', 'কে', 'কেউ', 'কেউই', 'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও', 'কোনো', 'ক্ষেত্রে', 'কয়েক', 'খুব', 'গিয়ে', 'গিয়েছে',
    'গিয়ে', 'গুলি', 'গেছে', 'গেল', 'গেলে', 'গোটা', 'চলে', 'চান', 'চায়', 'চার', 'চালু', 'চেয়ে', 'চেষ্টা', 'ছাড়া', 'ছাড়াও',
    'ছিল', 'ছিলেন', 'জন', 'জনকে', 'জনের', 'জন্য', 'জন্যওজে', 'জানতে', 'জানা', 'জানানো', 'জানায়', 'জানিয়ে', 'জানিয়েছে', 'জে',
    'জ্ঞান', 'টি', 'ঠিক', 'তখন', 'তত', 'তথা', 'তবু', 'তবে', 'তা', 'তাঁকে', 'তাঁদের', 'তাঁর', 'তাঁরা', 'তাঁাহারা', 'তাই', 'তাও',
    'তাকে', 'তাতে', 'তাতেই', 'তান', 'তার', 'তারপর', 'তারা', 'তারৈ', 'তারো', 'তাহলে', 'তাহা', 'তাহাতে', 'তাহার', 'তিনঐ', 'তিনি',
    'তিনিও', 'তুমি', 'তুলে', 'তেমন', 'তো', 'তোমার', 'থাকবে', 'থাকবেন', 'থাকা', 'থাকায়', 'থাকে', 'থাকেন', 'থেকে', 'থেকেই',
    'থেকেও', 'দিকে', 'দিতে', 'দিন', 'দিয়ে', 'দিয়েছে', 'দিয়েছেন', 'দিলেন', 'দু', 'দুই', 'দুটি', 'দুটো', 'দেওয়া', 'দেওয়ার',
    'দেওয়া', 'দেখতে', 'দেখা', 'দেখে', 'দেন', 'দেয়', 'দ্বারা', 'ধরা', 'ধরে', 'ধামার', 'নতুন', 'নয়', 'না', 'নাই', 'নাকি',
    'নাগাদ', 'নানা', 'নিজে', 'নিজেই', 'নিজেদের', 'নিজের', 'নিতে', 'নিয়ে', 'নিয়ে', 'নেই', 'নেওয়া', 'নেওয়ার', 'নেওয়া', 'নয়',
    'পক্ষে', 'পর', 'পরে', 'পরেই', 'পরেও', 'পর্যন্ত', 'পাওয়া', 'পাচ', 'পারি', 'পারে', 'পারেন', 'পি', 'পেয়ে', 'পেয়্র্', 'প্রতি',
    'প্রথম', 'প্রভৃতি', 'প্রযন্ত', 'প্রাথমিক', 'প্রায়', 'প্রায়', 'ফলে', 'ফিরে', 'ফের', 'বক্তব্য', 'বদলে', 'বন', 'বরং', 'বলতে',
    'বলল', 'বললেন', 'বলা', 'বলে', 'বলেছেন', 'বলেন', 'বসে', 'বহু', 'বা', 'বাদে', 'বার', 'বি', 'বিনা', 'বিভিন্ন', 'বিশেষ',
    'বিষয়টি', 'বেশ', 'বেশি', 'ব্যবহার', 'ব্যাপারে', 'ভাবে', 'ভাবেই', 'মতো', 'মতোই', 'মধ্যভাগে', 'মধ্যে', 'মধ্যেই', 'মধ্যেও',
    'মনে', 'মাত্র', 'মাধ্যমে', 'মোট', 'মোটেই', 'যখন', 'যত', 'যতটা', 'যথেষ্ট', 'যদি', 'যদিও', 'যা', 'যাঁর', 'যাঁরা', 'যাওয়া',
    'যাওয়ার', 'যাওয়া', 'যাকে', 'যাচ্ছে', 'যাতে', 'যাদের', 'যান', 'যাবে', 'যায়', 'যার', 'যারা', 'যিনি', 'যে', 'যেখানে', 'যেতে',
    'যেন', 'যেমন', 'র', 'রকম', 'রয়েছে', 'রাখা', 'রেখে', 'লক্ষ', 'শুধু', 'শুরু', 'সঙ্গে', 'সঙ্গেও', 'সব', 'সবার', 'সমস্ত',
    'সম্প্রতি', 'সহ', 'সহিত', 'সাধারণ', 'সামনে', 'সি', 'সুতরাং', 'সে', 'সেই', 'সেখান', 'সেখানে', 'সেটা', 'সেটাই', 'সেটাও',
    'সেটি', 'স্পষ্ট','?','-','♥️📖📚','একজন',"স্বয়ং",'অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত',
    'অন্য', 'অবধি', 'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে',
    'আজ', 'আদ্যভাগে', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের',
    'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি',
    'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি', 'একবার', 'একে',
    'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই',
    'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা',
    'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে',
    'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে',
    'করছেন', 'করতে', 'করবে', 'করবেন', 'করলে', 'করলেন', 'করা', 'করাই', 'করায়',
    'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই', 'করেছিলেন',
    'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে', 'কারও',
    'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই',
    'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও', 'কোনো', 'ক্ষেত্রে', 'কয়েক', 'খুব',
    'গিয়ে', 'গিয়েছে', 'গিয়ে', 'গুলি', 'গেছে', 'গেল', 'গেলে', 'গোটা', 'চলে',
    'চান', 'চায়', 'চার', 'চালু', 'চেয়ে', 'চেষ্টা', 'ছাড়া', 'ছাড়াও', 'ছিল',
                  
                  'ছিলেন', 'জন', 'জনকে', 'জনের', 'জন্য', 'জন্যওজে', 'জানতে', 'জানা', 'জানানো',
    'জানায়', 'জানিয়ে', 'জানিয়েছে', 'জে', 'জ্নজন', 'টি', 'ঠিক', 'তখন', 'তত',
    'তথা', 'তবু', 'তবে', 'তা', 'তাঁকে', 'তাঁদের', 'তাঁর', 'তাঁরা', 'তাঁাহারা',
    'তাই', 'তাও', 'তাকে', 'তাতে', 'তাহলে', 'তাহা', 'তাহাতে', 'তাহার', 'তিনঐ',
    'তিনি', 'তিনিও', 'তুমি', 'তুলে', 'তেমন', 'তো', 'তোমার', 'থাকবে', 'থাকবেন',
    'থাকা', 'থাকায়', 'থাকে', 'থাকেন', 'থেকে', 'থেকেই', 'থেকেও', 'দিকে', 'দিতে',
    'দিন', 'দিয়ে', 'দিয়েছে', 'দিয়েছেন', 'দিলেন', 'দু', 'দুই', 'দুটি', 'দুটো',
    'দেওয়া', 'দেওয়ার', 'দেওয়া', 'দেখতে', 'দেখা', 'দেখে', 'দেন', 'দেয়',
    'দ্বারা', 'ধরা', 'ধরে', 'ধামার', 'নতুন', 'নয়', 'না', 'নাই', 'নাকি', 'নাগাদ',
    'নানা', 'নিজে', 'নিজেই', 'নিজেদের', 'নিজের', 'নিতে', 'নিয়ে', 'নিয়ে', 'নেই',
    'নেওয়া', 'নেওয়ার', 'নেওয়া', 'নয়', 'পক্ষে', 'পর', 'পরে', 'পরেই', 'পরেও',
    'পর্যন্ত', 'পাওয়া', 'পাচ', 'পারি', 'পারে', 'পারেন', 'পি', 'পেয়ে', 'পেয়্র্',
    'প্রতি', 'প্রথম', 'প্রভৃতি', 'প্রযন্ত', 'প্রাথমিক', 'প্রায়', 'প্রায়', 'ফলে',
    'ফিরে', 'ফের', 'বক্তব্য', 'বদলে', 'বন', 'বরং', 'বলতে', 'বলল', 'বললেন', 'বলা',
    'বলে', 'বলেছেন', 'বলেন', 'বসে', 'বহু', 'বা', 'বাদে', 'বার', 'বি', 'বিনা',
    'বিভিন্ন', 'বিশেষ', 'বিষয়টি', 'বেশ', 'বেশি', 'ব্যবহার', 'ব্যাপারে', 'ভাবে',
    'ভাবেই', 'মতো', 'মতোই', 'মধ্যভাগে', 'মধ্যে', 'মধ্যেই', 'মধ্যেও', 'মনে', 'মাত্র',
    'মাধ্যমে', 'মোট', 'মোটেই', 'যখন', 'যত', 'যতটা', 'যথেষ্ট', 'যদি', 'যদিও', 'যা',
    'যাঁর', 'যাঁরা', 'যাওয়া', 'যাওয়ার', 'যাওয়া', 'যাকে', 'যাচ্ছে', 'যাতে',
    'যাদের', 'যান', 'যাবে', 'যায়', 'যার', 'যারা', 'যিনি', 'যে', 'যেখানে', 'যেতে',
    'যেন', 'যেমন', 'র', 'রকম', 'রয়েছে', 'রাখা', 'রেখে', 'লক্ষ', 'শুধু', 'শুরু',
    'সঙ্গে', 'সঙ্গেও', 'সব', 'সবার', 'সমস্ত', 'সহ', 'সহিত', 'সাধারণ', 'সামনে',
    'সি', 'সুতরাং', 'সে', 'সেই', 'সেখান', 'সেখানে', 'সেটা', 'সেটাই', 'সেটাও',
    'সেটি', 'স্পষ্ট', 'স্বয়ং', 'হইতে', 'হইবে', 'হইয়া','সে', 'সেই', 'সেখান', 'সেখানে', 'সেটা', 'সেটাই', 'সেটাও',
                      'সেটি', 'স্পষ্ট', 'স্বয়ং', 'হইতে', 'হইবে', 'হইয়া',
                      'এই', 'এক', 'এখান', 'এখানে', 'এইটা', 'এইটাই', 'এইটাও',
                      'এইটি', 'এটি', 'এত', 'এতে', 'এবং', 'এর', 'এরপর', 'এসে',
                      'এসেই', 'এসেছে', 'ও', 'করছে', 'করতে', 'করতেই', 'করতেইই',
                      'করতেইছে', 'করবে', 'করবেই', 'করবেইই', 'করবেছে', 'করবেন',
                      'করলে', 'করলেই', 'করলেইই', 'করলেছে', 'করলেন', 'করার', 'করি',
                      'করিতে','দেবে','দেবে','তারই','তার','করা', 'করে', 'করার', 'করলে', 'করার', 'করছি', 'করছে', 'করছেন', 'করবেন', 'করলি', 'করত', 'করতে', 'করাতে', 'করিয়া', 'করিতে', 'করিয়ে', 'করলো', 'করেছিল', 'করেছিলেন', 'করলেন', 'করারও', 'করারপর', 'করারআগে', 'করারজন্য', 'তার', 'তারপর', 'তারআগে', 'তারজন্য', 'তারপরও', 'তারআগেও', 'তারচেয়ে', 'তারবিশেষে', 'তিনি', 'তিনঐ', 'তিনিও', 'তাই', 'তবে', 'তখন', 'তবু', 'তাবু', 'তাতে', 'তবেও', 'তাতেও', 'তত', 'তাত', 'তথা', 'থাকে', 'থাকা', 'থাকতে', 'থাকার', 'থাকবে', 'থাকতেও', 'থাকবেও', 'থেকে', 'থেকেই', 'থেকেও', 'দু', 'দুটি', 'দুটো', 'দিন', 'দিতে', 'দিয়ে', 'দুটোও', 'দিকে', 'দুই', 'দেয়', 'দেওয়া', 'দেওয়ার', 'দেখতে', 'দেখা', 'দেখতেই', 'দেখার', 'দেখবে', 'দেখতেও', 'দেখবেও', 'দেখে', 'দেখলে', 'দেখলেই', 'দ্বারা', 'ধরা', 'ধরে', 'না', 'নাই', 'নাকি', 'নাগাদ', 'নানা', 'নিজে', 'নিজেই', 'নিজেও', 'নিজের', 'নিজেদের', 'নিজেদেরকে', 'নিজেদেরকেই', 'পর', 'পরে', 'পরেই', 'পরেও', 'পাওয়া', 'পারে', 'পারেন', 'পেয়ে', 'প্রতি', 'প্রথম', 'প্রযন্ত', 'প্রায়', 'প্রাথমিক', 'প্রায়', 'ফিরে', 'বরং', 'বলতে', 'বললেন', 'বলতেই', 'বললে', 'বলা', 'বলে', 'বলেছে', 'বলেছেন', 'বলেই', 'বলল', 'বলায়', 'বলেছিল', 'বলছে', 'বললে', 'বলতে', 'বলা', 'বলার', 'বহু', 'বা', 'বিশেষ', 'বেশ', 'বেশি', 'বললে', 'বলেছে', 'বলেছেন', 'বলেই', 'বলল', 'বলায়', 'বলেছিল', 'বলছে', 'বললে', 'বলতে', 'বলা', 'বলার', 'বহু', 'বা', 'বিশেষ', 'বেশ', 'বেশি', 'বসে', 'বলতে', 'বলতেই', 'বলতেও', 'বলে', 'বলেছে', 'বলেছিল', 'বলল', 'বলায়', 'বলার', 'বললেন', 'বলতে', 'বলা', 'বহু', 'বা', 'বিশেষ', 'বেশ', 'বেশি', 'ভাবে', 'ভাবেই', 'ভাবেও', 'ভাবেওই', 'মধ্যে', 'মধ্যেই', 'মধ্যেও', 'যখন',
                  
                  
                  'যা', 'যাই', 'যাওয়া', 'যাওয়ার', 'যে', 'যেখানে', 'যেতে', 'রকম', 'রয়েছে', 'রয়েছেন', 'শুধু', 'শুধুই', 'শুরু', 'সঙ্গে', 'সঙ্গেই', 'সহ', 'সহিত', 'সাথে', 'সুতরাং', 'সব', 'সবার', 'সবাই','আমি', 'তুমি', 'আপনি', 'সে', 'আমরা', 'তুমি', 'আপনি', 'তারা', 'আমার', 'তোমার', 'আপনার', 'তার', 'আমাদের', 'তোমাদের', 'আপনাদের', 'তাদের', 'এ', 'ও', 'কে','উপরে', 'নিচে', 'পাশে', 'মধ্যে', 'সাথে', 'থাকা', 'থেকে', 'দিকে', 'পরে', 'আগে', 'উপর', 'নিচে', 'সামনে', 'প্রতি', 'বরে', 'জন্য', 'বিরুদ্ধে', 'মুখোমুখি', 'দ্বারা', 'অবশ্যই','এবং', 'কিন্তু', 'তবে', 'তবু', 'যেন', 'কোনও', 'যদি', 'বা', 'অথবা', 'অথচ', 'কারণ', 'কেন', 'যাতে', 'যাতেই', 'হোক', 'যদিও', 'পরন্তু', 'ছাড়া', 'এমনকী', 'যতও', 'কিছুটা', 'যখন', 'যখনই', 'এই কারণে', 'এবং তাদের', 'তবে সে', 'তবু এটা', 'তার পরেও', 'তাতে', 'সোজা', 'এমন', 'যা কি', 'যাতে তারা', 'এই পর্যন্ত', 'এটি', 'হোকেও', 'সত্যিই', 'অথচ এটি', 'আপনি', 'তথাপি', 'অথবা সঙ্গে', 'আসলে', 'অতএব', 'যা কোনও', 'এখনও', 'তারা', 'আপনি এবং', 'এক্ষেত্রে', 'আমি', 'কেন না', 'আপনি এটি', 'আমরা যেভাবে', 'আমাদের কেউ', 'আপনি কি', 'অথবা তারা', 'তারা এবং', 'এবং আমরা', 'তাই', 'এটি আবার', 'তারা এবং আমি', 'আমি এবং', 'এটি এবং', 'অবশ্যই', 'এবং তারা', 'এবং তাদের', 'অথবা কোনও', 'অথবা তাদের', 'তাদের এবং', 'এবং তাদের', 'এবং একটি', 'আপনার পর', 'একই ভাবে', 'এবং এটি', 'এবং এই', 'এবং এটি', 'এবং এটা', 'এবং এটি', 'এবং একটি', 'এবং তারা', 'তারা এবং', 'এবং একটি', 'এবং এই', 'এবং এটি', 'তাদের সঙ্গে', 'এবং একটি', 'তারা এবং', 'এবং একটি', 'এবং তারা', 'তারা এবং', 'তারা এবং', 'এবং একটি', 'এবং তাদের', 'তাদের এবং', 'এবং একটি', 'এবং তারা', 'এবং একটি', 'এবং এটি', 'এবং একটি', 'এবং একটি', 'এবং এটি', 'এবং তারা', 'তারা এবং', 'এবং এটি', 'এবং এটি', 'এবং একটি', 'এবং তাদের', 'তাদের এবং', 'এবং একটি', 'এবং তারা', 'তারা এবং', 'এবং একটি', 'এবং তাদের', 'এবং একটি', 'এবং তারা', 'তারা এবং', 'এবং একটি', 'এবং একটি', 'এবং একটি', 'এবং তাদের',
                       'বই', 'কবিতা', 'উপন্যাস', 'গল্প', 'ছবি', 'কাহিনী', 'লেখা', 'লেখক', 'পুস্তক', 'প্রথম', 'বাংলা','একটি', 'অনেক', 'কিছু']




def remove_words(text):
    words = text.split()
    words = [word for word in words if word not in words_to_remove]
    return ' '.join(words)


df['summary'] = df['summary'].apply(remove_words)

In [ ]:
df['summary'] = df['summary'].apply(remove_punctuation)

In [ ]:
def data_summary(df):
    
    """
    This function will print the summary of the headlines and words distribution in the dataset. 
    
    Args:
        dataset: list of cleaned sentences   
        
    Returns:
        Number of documnets per class: int 
        Number of words per class: int
        Number of unique words per class: int
    """
    documents = []
    words = []
    u_words = []
    total_u_words = [word.strip().lower() for t in list(df.summary) for word in t.strip().split()]
    class_label= [k for k,v in df.genre.value_counts().to_dict().items()]
  
    for label in class_label: 
        word_list = [word.strip().lower() for t in list(df[df.genre==label].summary) for word in t.strip().split()]
        counts = dict()
        for word in word_list:
                counts[word] = counts.get(word, 0)+1
        
        ordered = sorted(counts.items(), key= lambda item: item[1],reverse = True)
        
        documents.append(len(list(df[df.genre==label].summary)))
    
        words.append(len(word_list))
        
        u_words.append(len(np.unique(word_list)))
       
        print("\nClass Name : ",label)
        print("Number of Documents:{}".format(len(list(df[df.genre==label].summary))))  
        print("Number of Words:{}".format(len(word_list))) 
        print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
        print("Most Frequent Words:\n")
        for k,v in ordered[:20]:
              print("{}\t{}".format(k,v))
    print("Total Number of Unique Words:{}".format(len(np.unique(total_u_words))))           
   
    return documents,words,u_words,class_label



documents,words,u_words,class_names = data_summary(df)

In [ ]:
df.summary[0]

In [ ]:
df.head()

In [ ]:
len(df.summary[0])

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['summary'])

In [ ]:
sequences = tokenizer.texts_to_sequences(df['summary'])

In [ ]:
maxlen = 4000
data = pad_sequences(sequences, maxlen=maxlen)

In [ ]:

labels = df['Label']


In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
import gensim.downloader as api

In [ ]:
word2vec_model = api.load("word2vec-google-news-300")

In [ ]:
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_dim = 300  # Word2Vec embeddings dimension
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]

In [ ]:
model = Sequential([
    Embedding(num_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(7, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout


model = Sequential([
    Embedding(num_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False),
    LSTM(128),  
    Dense(256, activation='relu'),
    Dropout(0.5),  
    Dense(7, activation='softmax')  
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
df.head()

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['Label'].unique()))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['summary'], df['Label'], test_size=0.2, random_state=42)


In [ ]:
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt')
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt')


In [ ]:
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())


In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
batch_size = 32


train_dataset = TensorDataset(X_train_tokens['input_ids'], X_train_tokens['attention_mask'], y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tokens['input_ids'], X_test_tokens['attention_mask'], y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for input_ids, attention_mask, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


In [ ]:
model.eval()
all_predictions = []
with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(dim=1)
        all_predictions.extend(predictions.tolist())

accuracy = accuracy_score(y_test_tensor, all_predictions)
print("Test Accuracy:", accuracy)